In [2]:
import matplotlib.pyplot as plt
import numpy as np
from qiskit import *
from qiskit.tools.visualization import plot_histogram   
from qiskit.providers.aer import QasmSimulator
from qiskit import transpile
import qiskit.quantum_info as qi

In [64]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, Aer, transpile, assemble
from qiskit.visualization import plot_histogram

# Physical parameters
Jmax = 2 * np.pi * 1.0
W = Jmax * 0 # change this for different disorder strengths
disorder = np.random.uniform(-W, W, 3)
B = Jmax * 4

# System parameters
alpha = 1.13
J = np.zeros((2, 2))
for i in range(2):
    for j in range(i+1, 2):
        J[i, j] = Jmax / (abs(i-j) ** alpha)
        J[j, i] = J[i, j]

dt = 0.01
total_time = 800 # change this for different evolution times

time_steps = int(total_time / dt)

# Create a quantum circuit
circ = QuantumCircuit(2, 2)
circ.h(range(2))

# Apply the time-evolution operator
for i in range(0, time_steps):
    circ.rx(dt*B, 0)
    circ.rx(dt*B, 1)
    circ.cx(0, 1)
    circ.rz(-2 * dt * J[0, 1], 1)
    circ.cx(0, 1)
    circ.rz(-2*dt*J[0, 1], 1)

    circ.rx(dt*W, 0)
    circ.rx(dt*W, 1)

circ.h(range(2))

circ.measure(range(2), range(2))  # Measure both spins

# Simulate
backend = Aer.get_backend('qasm_simulator')
circ_compiled = transpile(circ, backend)
job_sim = backend.run(circ_compiled, shots=1024)
result_sim = job_sim.result()

# Extract the counts from the result
counts = result_sim.get_counts(circ_compiled)

total_magnetization = 0
total_counts = sum(counts.values())

for outcome, count in counts.items():
    total_magnetization += count * (outcome.count('0') - outcome.count('1'))

expected_magnetization = total_magnetization / total_counts

# Normalize to [-1, 1]
normalized_expected_magnetization = expected_magnetization / len(counts)

# Plot the histogram
plot_histogram(counts)

# Print the normalized expected magnetization
print("Normalized Expected Magnetization (X basis):", normalized_expected_magnetization)


Normalized Expected Magnetization (X basis): 0.3427734375


In [66]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, Aer, transpile, assemble
from qiskit.visualization import plot_histogram

# Function to run the simulation for a given set of parameters
def run_simulation(W, total_time):
    # Physical parameters
    Jmax = 2 * np.pi * 1.0
    B = Jmax * 4

    # System parameters
    alpha = 1.13
    J = np.zeros((2, 2))
    for i in range(2):
        for j in range(i+1, 2):
            J[i, j] = Jmax / (abs(i-j) ** alpha)
            J[j, i] = J[i, j]

    dt = 0.01
    time_steps = int(total_time / dt)

    # Create a quantum circuit
    circ = QuantumCircuit(2, 2)
    circ.h(range(2))

    # Apply the time-evolution operator
    for i in range(time_steps):
        circ.rx(dt*B, 0)
        circ.rx(dt*B, 1)
        circ.cx(0, 1)
        circ.rz(-2 * dt * J[0, 1], 1)
        circ.cx(0, 1)
        circ.rz(-2*dt*J[0, 1], 1)

        circ.rx(dt*W, 0)
        circ.rx(dt*W, 1)

    circ.h(range(2))
    circ.measure(range(2), range(2))  # Measure both spins

    # Simulate
    backend = Aer.get_backend('qasm_simulator')
    circ_compiled = transpile(circ, backend)
    job_sim = backend.run(circ_compiled, shots=1024)
    result_sim = job_sim.result()

    # Extract the counts from the result
    counts = result_sim.get_counts(circ_compiled)

    total_magnetization = 0
    total_counts = sum(counts.values())

    for outcome, count in counts.items():
        total_magnetization += count * (outcome.count('0') - outcome.count('1'))

    expected_magnetization = total_magnetization / total_counts

    # Normalize to [-1, 1]
    normalized_expected_magnetization = expected_magnetization / len(counts)

    # Return normalized expected magnetization
    return normalized_expected_magnetization

# Case 1: W = 0
W_0 = 0
total_times = np.arange(100, 701, 100)
normalized_magnetizations_0 = []

for total_time in total_times:
    normalized_magnetization = run_simulation(W_0, total_time)
    normalized_magnetizations_0.append(normalized_magnetization)

# Case 2: W = 8 * Jmax
W_max = 8 * Jmax
normalized_magnetizations_max = []

for total_time in total_times:
    normalized_magnetization = run_simulation(W_max, total_time)
    normalized_magnetizations_max.append(normalized_magnetization)

# Plot the results on the same graph
plt.plot(total_times/100, normalized_magnetizations_0, label='No disorder')
plt.plot(total_times/100, normalized_magnetizations_max, label='W = 8 * Jmax')
plt.xlabel('Total Time')
plt.ylabel('Expected Magnetization')
plt.legend()
plt.show()


KeyboardInterrupt: 